In [1]:
# As usual, a bit of setup
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch
from scipy.special import softmax

from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.rnn_layers import *
from cs231n.captioning_solver import CaptioningSolver
from cs231n.classifiers.rnn import CaptioningRNN
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url
from cs231n.layers import *
from cs231n.rnn_layers import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
np.random.seed(231)

# build data and model
small_data = load_coco_data(max_train=50)
small_rnn_model = CaptioningRNN(
          cell_type='rnn',
          word_to_idx=small_data['word_to_idx'],
          input_dim=small_data['train_features'].shape[1],
          hidden_dim=512,
          wordvec_dim=256,
        )

# get minibatch of data
minibatch = sample_coco_minibatch(small_data,
                                  batch_size=25,
                                  split='train')

# build captions and mask
captions, features, urls = minibatch
captions_in = captions[:, :-1]
captions_out = captions[:, 1:]
mask = (captions_out != small_rnn_model._null)

# unpack weights
W_proj, b_proj = small_rnn_model.params['W_proj'], small_rnn_model.params['b_proj']
W_embed = small_rnn_model.params['W_embed']
Wx, Wh, b = small_rnn_model.params['Wx'], small_rnn_model.params['Wh'], small_rnn_model.params['b']
W_vocab, b_vocab = small_rnn_model.params['W_vocab'], small_rnn_model.params['b_vocab']
loss, grads = 0.0, {}

In [3]:
# forward pass up to softmax loss
affine_out, affine_cache = affine_forward(features, W_proj, b_proj)
captions_in_embed, embed_cache = word_embedding_forward(captions_in, W_embed)
hidden_rnn, rnn_cache = rnn_forward(captions_in_embed, affine_out, Wx, Wh, b)
scores, scores_cache = temporal_affine_forward(hidden_rnn, W_vocab, b_vocab)
loss, dscores = temporal_softmax_loss(scores, captions_out, mask)

In [4]:
loss

80.14522802655904

## (4) (temporal) affine transformation

This is just an affine transformation but for sequence of values.

In [6]:
dhidden_rnn, grads['W_vocab'], grads['b_vocab'] = \
temporal_affine_backward(dscores, scores_cache)

## (3) rnn layer

## (2) word embedding layer

## (1) affine transformation